In [1]:
from langchain_core.tools import tool
import httpx
from geopy.geocoders import Nominatim # uv add geopy
import json

def get_coordinates(city_name: str) -> tuple[float, float]:
    """도시 이름을 받아 위도와 경도를 반환합니다."""
    geolocator = Nominatim(user_agent="weather_app_langgraph")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    raise ValueError(f"좌표를 찾을 수 없습니다: {city_name}")

@tool
def get_weather(city_name: str) -> str:
    """도시 이름을 받아 해당 도시의 현재 날씨 정보를 반환합니다."""
    print(f"날씨 조회: {city_name}")
    latitude, longitude = get_coordinates(city_name)
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = httpx.get(url)
    response.raise_for_status()
    return json.dumps(response.json())

tools = [get_weather]

In [2]:
get_coordinates("부산")

(35.1799528, 129.0752365)

In [3]:
tools[0].invoke({"city_name": "부산"})

날씨 조회: 부산


'{"latitude": 35.2, "longitude": 129.0625, "generationtime_ms": 0.05078315734863281, "utc_offset_seconds": 0, "timezone": "GMT", "timezone_abbreviation": "GMT", "elevation": 26.0, "current_weather_units": {"time": "iso8601", "interval": "seconds", "temperature": "\\u00b0C", "windspeed": "km/h", "winddirection": "\\u00b0", "is_day": "", "weathercode": "wmo code"}, "current_weather": {"time": "2025-09-27T08:45", "interval": 900, "temperature": 23.8, "windspeed": 5.6, "winddirection": 63, "is_day": 1, "weathercode": 1}}'

In [2]:
from langgraph.prebuilt import ToolNode # 에이전트와 도구 간의 인터페이스 역할을 하며, 언어 모델의 지시에 따라 실제 도구 실행을 담당하는 핵심적인 중간 다리
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash").bind_tools(tools)
tool_node = ToolNode([get_weather])

def call_model(state: MessagesState):
    return {"messages": [model.invoke(state["messages"])]}

graph_builder = StateGraph(MessagesState)
graph_builder.add_node("call_model", call_model)
graph_builder.add_node("tool_node", tool_node)

graph_builder.add_edge(START, "call_model")
graph_builder.add_conditional_edges(
    "call_model",
    lambda s: "tool_node" if s["messages"][-1].tool_calls else END,
    {"tool_node": "tool_node", END: END},
)
graph_builder.add_edge("tool_node", "call_model")

graph = graph_builder.compile()

In [3]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
# [venv 경로]\Lib\site-packages\langchain_core\runnables\graph_mermaid.py
# 파일을 열고 `_render_mermaid_using_pyppeteer` 함수 내부에 있는 `pyppeteer.launch()` 호출 부분을 찾아 `executablePath` 인자를 추가한다.
# browser = await launch(executablePath=r'C:\Program Files\Google\Chrome\Application\chrome.exe')
import nest_asyncio
from IPython.display import display, Image
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

# nest_asyncio를 적용하여 이벤트 루프 중첩 오류를 해결한다.
nest_asyncio.apply()

# (옵션) 그래프를 시각화
# 이 코드는 이미 langchain_core 라이브러리의 pyppeteer.launch()에 
# executablePath가 하드코딩된 상태를 전제로 한다.
display(Image(graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))

In [4]:
graph.get_graph().print_ascii()

          +-----------+             
          | __start__ |             
          +-----------+             
                 *                  
                 *                  
                 *                  
          +------------+            
          | call_model |            
          +------------+            
          ...         ***           
         .               *          
       ..                 **        
+---------+           +-----------+ 
| __end__ |           | tool_node | 
+---------+           +-----------+ 


In [5]:
from langchain_core.messages import HumanMessage

query = "부산 날씨 어때?"
result = graph.invoke({"messages": [HumanMessage(content=query)]})

result

날씨 조회: 부산


{'messages': [HumanMessage(content='부산 날씨 어때?', additional_kwargs={}, response_metadata={}, id='23c165cf-ae96-4539-87ec-930e1e22adbf'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city_name": "\\ubd80\\uc0b0"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--a1202c97-7dcb-49fa-9171-a42e6d4c5dff-0', tool_calls=[{'name': 'get_weather', 'args': {'city_name': '부산'}, 'id': '0b3efad4-8132-4dbf-9d77-076f08862c3c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 81, 'total_tokens': 138, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 64}}),
  ToolMessage(content='{"latitude": 35.2, "longitude": 129.0625, "generationtime_ms": 0.06592273712158203, "utc_offset_seconds": 0, "timezone": "GMT", "timezone_abbreviation": "GMT", "elevation": 26.0, "current_weather

In [6]:
result['messages'][-1].content

'부산의 현재 날씨는 23.3°C이며, 맑음(weathercode 2)입니다. 바람은 시속 10.3km/h로 남서쪽(234°)에서 불고 있습니다.'

In [9]:
from langchain_core.messages import HumanMessage

query = "랭그래프에 대해 핵심만 간단히 설명해줘."
result = graph.invoke({"messages": [HumanMessage(content=query)]})

result

{'messages': [HumanMessage(content='랭그래프에 대해 핵심만 간단히 설명해줘.', additional_kwargs={}, response_metadata={}, id='9a81786a-25f2-4863-acb0-0b7a4bf05a63'),
  AIMessage(content='랭그래프는 복잡한 LLM(대규모 언어 모델) 애플리케이션을 구축하기 위한 라이브러리입니다. 여러 LLM 호출, 도구 사용, 그리고 사람의 개입까지 포함하는 다단계 프로세스를 그래프 형태로 정의하고 실행할 수 있게 해줍니다. 이를 통해 더 정교하고 상태를 가지는 LLM 에이전트나 워크플로우를 만들 수 있습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--90ba5a5e-2c62-4f0b-94c0-4fc9df193abe-0', usage_metadata={'input_tokens': 64, 'output_tokens': 136, 'total_tokens': 200, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 48}})]}

In [8]:
print(result['messages'][-1].content)

랭체인은 대규모 언어 모델(LLM)을 활용하여 애플리케이션을 개발할 수 있도록 돕는 프레임워크입니다. LLM 기반의 복잡한 워크플로우를 쉽게 구성하고, 외부 데이터 소스와 연동하며, 다양한 구성 요소를 연결하여 더 강력한 애플리케이션을 만들 수 있게 해줍니다.
